<a href="https://colab.research.google.com/github/mebalint/hgt/blob/main/hgt_football_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Predicting football match results with machine learning




### Importing packages

In [3]:
import pandas as pd
#from segedfgek import *
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.model_selection import train_test_split

#from SBS import *
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.gaussian_process.kernels import RBF

#from PlotDecisionRegions import *
from sklearn.decomposition import PCA

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import KernelPCA

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier

from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso, LassoLars, Ridge, BayesianRidge, SGDRegressor, LinearRegression
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor

from sklearn.model_selection import RandomizedSearchCV

# to supress warnings
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1)

### Data preparation
A https://figshare.com/collections/Soccer_match_event_dataset/4415000 címen nyilvánosan elérhető adatból nyertük ki saját számításaink alapján a számunkra fontos mennyiségeket. Az adat az európai top 5 futball bajnokság (angol, német, olasz, spanyol, francia) 2017/18-as szezonjának összes meccséről szól, egy adott meccsen minden labdás eseményről van feljegyzés (pl. passzok, lövések, gólok). Ezen események alapján az egyszerű meccsstatisztikákat (pl. kapura lövés, szabadrúgások) és néhány hálózattudományi mértéket (pl. betweenness centrality) határoztunk meg.

In [2]:
matches=pd.read_csv('https://raw.githubusercontent.com/mebalint/hgt/main/matches_final.csv', sep='\t', index_col=0)
matches

,home,away,home_goal,away_goal,roundId,gameweek,competitionId,home_corner,away_corner,home_pass,...,away_possession,home_degree,away_degree,home_diameter,away_diameter,home_bet_cent,away_bet_cent,home_entropy,away_entropy,match_id
0,Lazio,Inter,2,3,4406278,38,524,10,3,302,...,0.60,22.000,37.545,2,2,0.027,0.019,0.072,0.032,2576335
1,Sassuolo,AS Roma,0,1,4406278,38,524,5,9,337,...,0.58,23.182,36.909,2,2,0.024,0.011,0.066,0.032,2576336
2,Cagliari,Atalanta,1,0,4406278,38,524,5,11,232,...,0.69,15.545,45.909,2,2,0.037,0.019,0.111,0.027,2576329
3,Chievo,Benevento,1,0,4406278,38,524,2,4,198,...,0.76,12.818,55.364,2,2,0.042,0.025,0.144,0.021,2576330
4,Udinese,Bologna,1,0,4406278,38,524,10,2,327,...,0.59,24.636,39.636,3,2,0.033,0.019,0.064,0.032,2576331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,Tottenham,Swansea,0,0,4405654,5,364,11,1,753,...,0.25,60.636,13.909,2,3,0.013,0.037,0.018,0.130,2499766
1822,Tottenham,Burnley,1,1,4405654,3,364,10,7,573,...,0.34,44.909,18.545,2,2,0.016,0.026,0.026,0.086,2499746
1823,Tottenham,Chelsea,1,2,4405654,2,364,14,3,569,...,0.32,43.545,17.091,2,2,0.009,0.030,0.025,0.098,2499737
1824,Sevilla,Alaves,1,0,4406122,38,795,8,3,601,...,0.39,48.091,25.364,2,3,0.013,0.030,0.024,0.057,2565920
